## Rule Induction

In [1]:
#import pandas as pd
#import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
#import pickle

In [2]:
import Orange

### Risk data

In [6]:
risk_data = Orange.data.Table('rule_risk_data.csv')

In [10]:
risk_data

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

In [11]:
risk_data.domain

[historia, divida, garantias, renda | risco]

In [12]:
import Orange.classification


cn2 = Orange.classification.rules.CN2Learner()
rules = cn2(risk_data)

In [13]:
for rule in rules.rule_list:
    print(rule)

IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=baixo 
IF TRUE THEN risco=alto 


In [15]:
#case1: history: good, debt: high, guarantee: none, income: >35
#                0            0                 1           2

#case2: history: bad, debt: high, guarantee: adequate, income: <15
#                2            0                 0           0

prediction = rules([['boa','alta','nenhuma','acima_35'],
                   ['ruim','alta','adequada','0_15']])
prediction

array([1, 0], dtype=int64)

In [17]:
risk_data.domain.class_var.values

('alto', 'baixo', 'moderado')

In [19]:
for i in prediction:
    print(risk_data.domain.class_var.values[i])

baixo
alto


### Credit data

In [20]:
credit_data = Orange.data.Table('rule_credit_data.csv')

In [22]:
credit_data.domain , credit_data

([income, age, loan | default],
 [[66155.9, 59.017, 8106.53 | 0],
  [34415.2, 48.1172, 6564.75 | 0],
  [57317.2, 63.108, 8020.95 | 0],
  [42709.5, 45.752, 6103.64 | 0],
  [66952.7, 18.5843, 8770.1 | 1],
  ...
 ])

In [23]:
import Orange.evaluation


split_data = Orange.evaluation.testing.sample(credit_data, n = 0.25)

In [24]:
split_data

([[41994, 36.1928, 6644.34 | 0],
  [42889.3, 57.5067, 6340.71 | 0],
  [62724.6, 39.8354, 9255.14 | 0],
  [34796, 58.4879, 443.667 | 0],
  [29672.6, 51.0198, 607.909 | 0],
  ...
 ],
 [[55704.8, 48.5166, 5594.33 | 0],
  [56256, 22.1605, 5452.24 | 0],
  [67730.4, 26.3031, 8881.58 | 1],
  [42750, 56.4191, 4626.54 | 0],
  [26181.2, 42.663, 2618.97 | 0],
  ...
 ])

In [26]:
credit_data_test = split_data[0]
credit_data_training = split_data[1]

In [27]:
len(credit_data_test), len(credit_data_training)

(500, 1500)

In [28]:
cn2 = Orange.classification.rules.CN2Learner()
rules = cn2(credit_data_training)

In [29]:
for rule in rules.rule_list:
    print(rule)

IF age>=34.9257164876908 THEN default=0 
IF loan<=2495.13299137587 AND income>=20145.9885970689 THEN default=0 
IF loan<=4265.17370408717 AND income>=30075.264924671497 THEN default=0 
IF loan>=7708.31562497011 AND loan>=9698.582169129 THEN default=1 
IF loan>=7708.31562497011 AND loan>=9595.28628892989 THEN default=0 
IF loan>=7708.31562497011 AND age>=21.4227129220963 THEN default=1 
IF income>=57787.565658800304 AND loan>=8092.982779946221 THEN default=1 
IF income>=57787.565658800304 THEN default=0 
IF loan>=6241.27050834808 THEN default=1 
IF loan<=4665.58102065705 AND income>=42990.9828281961 THEN default=0 
IF income>=42236.4560933784 AND income>=50527.5841732509 THEN default=0 
IF loan>=5836.56338145928 THEN default=1 
IF income>=42236.4560933784 AND income>=45971.133494749694 THEN default=0 
IF age>=27.441609795671 AND income>=30187.0918646455 THEN default=1 
IF loan>=3105.4430213977303 AND age>=27.441609795671 THEN default=1 
IF age<=24.3714581453241 AND age>=24.3714581453241

In [33]:
prediction = Orange.evaluation.testing.TestOnTestData(credit_data_training,
                                                      credit_data_test,
                                                      [lambda testdata: rules])
prediction

In [34]:
Orange.evaluation.CA(prediction) #classification accuracy (ca)

array([0.982])